In [1]:
import pandas as pd
df = pd.read_csv('data/Consumer_Complaints.csv')
df.info()

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,6,11,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1298835 entries, 0 to 1298834
Data columns (total 18 columns):
Date received                   1298835 non-null object
Product                         1298835 non-null object
Sub-product                     1063669 non-null object
Issue                           1298835 non-null object
Sub-issue                       765598 non-null object
Consumer complaint narrative    384314 non-null object
Company public response         458244 non-null object
Company                         1298835 non-null object
State                           1278830 non-null object
ZIP code                        1181648 non-null object
Tags                            177850 non-null object
Consumer consent provided?      697709 non-null object
Submitted via                   1298835 non-null object
Date sent to company            1298835 non-null object
Company response to consumer    1298828 non-null object
Timely response?                1298835 non-null obj

In [2]:
feature_col = ['Consumer complaint narrative']
res_col = ['Product', 'Issue']

In [3]:
df.dropna(subset= feature_col + res_col, inplace=True)
df.drop_duplicates(subset=feature_col, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 367629 entries, 44142 to 929033
Data columns (total 18 columns):
Date received                   367629 non-null object
Product                         367629 non-null object
Sub-product                     317356 non-null object
Issue                           367629 non-null object
Sub-issue                       254272 non-null object
Consumer complaint narrative    367629 non-null object
Company public response         173475 non-null object
Company                         367629 non-null object
State                           366264 non-null object
ZIP code                        281945 non-null object
Tags                            64498 non-null object
Consumer consent provided?      367629 non-null object
Submitted via                   367629 non-null object
Date sent to company            367629 non-null object
Company response to consumer    367625 non-null object
Timely response?                367629 non-null object
Consum

In [ ]:
#print(df['Product'].unique())

In [4]:
df_cat = None
for col in res_col:
    temp = df[col].astype('category')
    df_cat = pd.concat([df_cat, temp], axis=1)

df.drop(columns=res_col, inplace=True)
df = pd.concat([df, df_cat], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 367629 entries, 44142 to 929033
Data columns (total 18 columns):
Date received                   367629 non-null object
Sub-product                     317356 non-null object
Sub-issue                       254272 non-null object
Consumer complaint narrative    367629 non-null object
Company public response         173475 non-null object
Company                         367629 non-null object
State                           366264 non-null object
ZIP code                        281945 non-null object
Tags                            64498 non-null object
Consumer consent provided?      367629 non-null object
Submitted via                   367629 non-null object
Date sent to company            367629 non-null object
Company response to consumer    367625 non-null object
Timely response?                367629 non-null object
Consumer disputed?              160949 non-null object
Complaint ID                    367629 non-null int64
Product

In [ ]:
# print(df['Issue'].unique())

In [11]:
# randomly select train/test data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[feature_col[0]], df[res_col[0]], test_size=0.2, random_state=42)

In [13]:
y_train.head()
#X_train.head()

866293    I received a collection document from Dynamic ...
885048    Attempting to dispute a charge and the credit ...
745921    I was approved for a capital one credit card X...
90237     My mortgage company ( Fairway Mortgage ) sold ...
402152                open accounts cra and hippa violation
Name: Consumer complaint narrative, dtype: object

In [7]:
import nltk
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora

[nltk_data] Downloading package wordnet to /Users/yunfei/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['xxxx', 'xx'])
stop_words = set(stop_words)

In [9]:
def tokenize(doc):
    # doc is a string
    # return an array of words
    return simple_preprocess(doc, deacc=True, min_len=2, max_len=15)

def rm_stopwords_and_lemmatize(token_array, flag_rm_stop=True, flag_lemmatize=True):
    out = []
    for token in token_array:
        if flag_lemmatize:
            token = WordNetLemmatizer().lemmatize(token)
        if flag_rm_stop:
            if token not in stop_words:
                out.append(token)
        else:
            out.append(token)
    return out

def my_tokenizer(doc, flag_rm_stop=True, flag_lemmatize=True):
    return rm_stopwords_and_lemmatize(tokenize(doc), flag_rm_stop, flag_lemmatize)

In [ ]:
#text = 'I struggled so much with the settings.'
#tokens = tokenize(text)
#print(tokens)
#print(rm_stopwords_and_lemmatize(tokens))
#print(rm_stopwords_and_lemmatize(tokens, flag_rm_stop=False))
#print(rm_stopwords_and_lemmatize(tokens, flag_lemmatize=False))
#print(rm_stopwords_and_lemmatize(tokens, flag_rm_stop=False, flag_lemmatize=False))
#print(my_tokenizer(text))

In [21]:
#define vectorizer parameters
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=1000,
                                   min_df=5, 
                                   stop_words='english',
                                   use_idf=True, 
                                   tokenizer=my_tokenizer, 
                                   token_pattern=r"\b\w[\w']+\b",
                                   ngram_range=(1,2))

tfidf_matrix = tfidf_vectorizer.fit_transform(X_train) #fit the vectorizer to corpus (min = 0.0, max = 1.0)

print (tfidf_matrix.shape)

(294103, 1000)


In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
#clf = RandomForestClassifier()

In [28]:
from sklearn.model_selection import cross_val_score

acc = cross_val_score(clf, tfidf_matrix, y_train, scoring='accuracy', cv=5)
print(acc)

[0.6764066  0.67637909 0.67902683 0.67967355 0.67703817]


In [16]:
#model = clf.fit(tfidf_matrix, y_train)

In [22]:
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 20
lda = LatentDirichletAllocation(n_components=n_topics, 
                                learning_method='online')

In [23]:
tfidf_matrix_lda = (tfidf_matrix * 100)
tfidf_matrix_lda = tfidf_matrix_lda.astype(int)

In [24]:
lda.fit(tfidf_matrix_lda)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=20, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [25]:
print(lda.components_.shape)

(20, 1000)


In [26]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

pyLDAvis.sklearn.prepare(lda, tfidf_matrix_lda, tfidf_vectorizer)

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
5      9.619506        1       1 -0.244005 -0.168414
14     8.182956        1       2  0.151158  0.137409
0      7.875827        1       3  0.178024  0.126633
10     7.625511        1       4  0.090454  0.121697
18     6.015752        1       5  0.219480 -0.031527
12     5.723387        1       6  0.167570 -0.068014
8      5.535933        1       7  0.061247 -0.128848
7      5.361076        1       8 -0.146499  0.104852
9      5.348170        1       9 -0.118567  0.107417
6      5.090265        1      10 -0.083349  0.190753
11     4.939698        1      11 -0.111844  0.162832
15     4.661760        1      12  0.082462 -0.184188
19     4.320424        1      13  0.006842 -0.259255
13     3.741443        1      14 -0.192660  0.057400
4      2.977970        1      15 -0.205821 -0.124840
3      2.956884        1      16  0.089960 -0.005397
1      2.764966        1      17 -0.113392 -0.032166
17     2.633569        1      18 -0.011570  0.069146
16     2.470596        1      19  0.080538 -0.026291
2      2.154310        1      20  0.099971 -0.049199, topic_info=     Category          Freq           Term         Total  loglift  logprob
term                                                                      
650   Default  1.369973e+06        payment  1.369973e+06  30.0000  30.0000
263   Default  1.132498e+06           debt  1.132498e+06  29.0000  29.0000
524   Default  1.210220e+06           loan  1.210220e+06  28.0000  28.0000
133   Default  8.440730e+05           card  8.440730e+05  27.0000  27.0000
7     Default  1.812697e+06        account  1.812697e+06  26.0000  26.0000
232   Default  1.853091e+06         credit  1.853091e+06  25.0000  25.0000
766   Default  1.245546e+06         report  1.245546e+06  24.0000  24.0000
502   Default  6.964830e+05         letter  6.964830e+05  23.0000  23.0000
194   Default  9.073460e+05        company  9.073460e+05  22.0000  22.0000
184   Default  6.932900e+05     collection  6.932900e+05  21.0000  21.0000
239   Default  8.741300e+05  credit report  8.741300e+05  20.0000  20.0000
93    Default  8.601240e+05           bank  8.601240e+05  19.0000  19.0000
996   Default  6.087920e+05           year  6.087920e+05  18.0000  18.0000
458   Default  5.179170e+05        inquiry  5.179170e+05  17.0000  17.0000
452   Default  8.803180e+05    information  8.803180e+05  16.0000  16.0000
606   Default  5.988570e+05         number  5.988570e+05  15.0000  15.0000
641   Default  6.397800e+05           paid  6.397800e+05  14.0000  14.0000
154   Default  4.894540e+05         charge  4.894540e+05  13.0000  13.0000
484   Default  5.198760e+05           late  5.198760e+05  12.0000  12.0000
772   Default  6.297010e+05      reporting  6.297010e+05  11.0000  11.0000
91    Default  5.435850e+05        balance  5.435850e+05  10.0000  10.0000
236   Default  4.416390e+05    credit card  4.416390e+05   9.0000   9.0000
159   Default  4.498370e+05          check  4.498370e+05   8.0000   8.0000
321   Default  4.878540e+05        equifax  4.878540e+05   7.0000   7.0000
356   Default  5.015500e+05            fee  5.015500e+05   6.0000   6.0000
331   Default  3.468930e+05       experian  3.468930e+05   5.0000   5.0000
580   Default  6.134870e+05       mortgage  6.134870e+05   4.0000   4.0000
379   Default  3.140350e+05          fraud  3.140350e+05   3.0000   3.0000
777   Default  3.820200e+05        request  3.820200e+05   2.0000   2.0000
99    Default  2.807110e+05     bankruptcy  2.807110e+05   1.0000   1.0000
...       ...           ...            ...           ...      ...      ...
672   Topic20  1.546043e+05           plan  1.546053e+05   3.8377  -3.0180
542   Topic20  1.170645e+05         mailed  1.170655e+05   3.8377  -3.2962
441   Topic20  1.076922e+05       included  1.076932e+05   3.8377  -3.3796
294   Topic20  9.393392e+04       discover  9.393487e+04   3.8377  -3.5163
153   Topic20  8